In [18]:
import os
import torch
import cv2
import math
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import datasets, models, transforms
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
import datetime
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
from PIL import Image
from tqdm import tqdm
from torchvision.models import VGG16_Weights, efficientnet_v2_m, EfficientNet_V2_M_Weights
import time
from collections import deque
from IPython.display import clear_output
from feature_extraction import *
from DSTA import *
plt.ion()

In [19]:
from ultralytics import YOLO

In [20]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [21]:
def draw_probability_bar(frame, probability, position=(50, 50), bar_width=100, bar_height=10):
    bar_length = int(bar_width * probability)
    end_point = (position[0] + bar_length, position[1] + bar_height)
    cv2.rectangle(frame, position, end_point, (0, 255, 0), -1)

In [22]:
object_detector = YOLO('yolov9e.pt')  # load an official model
object_detector = YOLO('best.pt')

In [23]:
classNames = ['Bus', 'Car', 'Truck', 'Motorcycle', 'Pedestrain', 'Truck']

In [24]:
feature_extractor = init_feature_extractor(backbone='efficientnet', device=device)

In [25]:
dsta = DSTA(device=torch.device('mps'), d=512, m=5, input_dim=512+512, output_dim=2, n_layers=1, t=1, n_objects=12).to(torch.float32)
dsta.load_state_dict(torch.load('best_dsta.pth')['dsta_state_dict'])
dsta.eval()

TypeError: BFloat16 is not supported on MPS

In [9]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((640, 640)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:
with torch.no_grad():
        cap = cv2.VideoCapture(0)
        cap.set(3, 640)
        while cap.isOpened():
            ret, frame = cap.read()
            '''
            results = object_detector(frame, stream=True)

            # coordinates
            for r in results:
                boxes = r.boxes

                for box in boxes:
                    # bounding box
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                    # put box in cam
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)

                    # confidence
                    confidence = math.ceil((box.conf[0]*100))/100
                    print("Confidence --->",confidence)

                    # class name
                    cls = int(box.cls[0])
                    print("Class name -->", classNames[cls])

                    # object details
                    org = [x1, y1]
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    fontScale = 1
                    color = (255, 0, 0)
                    thickness = 2

                    cv2.putText(frame, classNames[cls], org, font, fontScale, color, thickness)
                    '''
            frame_tensor = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_tensor = Image.fromarray(frame_tensor)
            frame_tensor = transform(frame_tensor)


            out, h, t_pred = DSTA(frame_tensor.unsqueeze(0), torch.tensor([[0, 1]]), with_tsaa=False)
            draw_probability_bar(frame, out[:,1])
            

            cv2.imshow('Webcam', frame)
            if cv2.waitKey(1) == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

2024-05-05 11:43:13.809 python[76643:9929203] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
